In [ ]:
import ikpy
import sys
import numpy as np
from ikpy import plot_utils
from os.path import abspath
from pypot.creatures import PoppyTorso

sys.path.append('./sketchModel')
from sketch_pix2seq_sampling import getAbsoluteStrokes,drawAbsolutePosition

In [ ]:
strokes = getAbsoluteStrokes('','','./sketchModel/outputs/snapshot','./200.png')

In [ ]:
#scene_path = abspath('./vrep-sensors/torso_sensors_scene.ttt')
#config_path = abspath('./vrep-sensors/torso_vrep_sensors_config.json')
config_path = abspath('./config/torso_config_cam01.json')
#poppy = PoppyTorso(simulator='vrep', scene=scene_path, config=config_path)
poppy = PoppyTorso(config=config_path)

In [ ]:
for motor in poppy.motors:
    print(motor.name)
    motor.compliant = True
    motor.goto_position(0, 3)

In [ ]:
poppy.bust_y.compliant = True

In [ ]:
poppy.r_elbow_y.goto_position(10.9,0.5)
poppy.r_arm_z.goto_position(-45,0.5) #Angle, Time

In [ ]:
drawAbsolutePosition(strokes,(0,48,0,48))

In [ ]:
image_size = 0.10
def rescaleImage(strokes,init_size,end_size,xFactor, yFactor):
    new_stroke = []
    for stroke in strokes:
        x = end_size/(init_size/stroke[0])
        y = end_size/(init_size/stroke[1])
        new_stroke.append([x,y,stroke[2]])
    new_stroke = np.asarray(new_stroke)
    
    xMin = np.min(new_stroke[:,0])
    xMax = np.max(new_stroke[:,0])

    yMin = np.min(new_stroke[:,1])
    yMax = np.max(new_stroke[:,1])
    
    
    multiX = end_size/(xMax-xMin)
    multiY = end_size/(yMax-yMin)
    
    new_stroke[:,0] = new_stroke[:,0] * np.floor(multiX)
    new_stroke[:,1] = new_stroke[:,1] * np.floor(multiY)
    new_stroke = moveImage(new_stroke,xFactor,yFactor)
    return new_stroke

def moveImage(stroke_rescale, xFactor, yFactor):
    xMin = np.min(stroke_rescale[:,0])
    yMin = np.min(stroke_rescale[:,1])

    stroke_rescale[:,0] = stroke_rescale[:,0] - (xMin * xFactor)
    stroke_rescale[:,1] = stroke_rescale[:,1] - (yMin * yFactor)
    return stroke_rescale

stroke_rescale = rescaleImage(strokes, 48, image_size,0.98, 0.98)

xMin = np.min(stroke_rescale[:,0])
xMax = np.max(stroke_rescale[:,0])

yMin = np.min(stroke_rescale[:,1])
yMax = np.max(stroke_rescale[:,1])
    
drawAbsolutePosition(stroke_rescale,(0,image_size+0.01,0,image_size+0.01))


In [ ]:
'''import numpy as np
r = .13
x0, y0, z0 = (0.2, -0.2, 0.2)

poppy.l_arm_chain.goto((x0, y0, z0), 1., wait=True)


for alpha in np.arange(0, 4*np.pi, .08):
    x = r * np.cos(alpha) + x0
    z = r * np.sin(alpha) + z0
    
    poppy.l_arm_chain.goto((x, y0, z), 0.03, wait=True)'''

In [ ]:
poppy.l_arm_chain.goto((0.10, -0.20, 0.10), 0.5, wait=True)


In [ ]:
def drawWithPoppy(planeY, withoutDraw):
    axisY = planeY
    for stroke in stroke_rescale:
        if stroke[2] == 0:
            axisY = planeY
        else:
            axisY = withoutDraw
        poppy.l_arm_chain.goto((stroke[0], axisY, stroke[1]), 0.5, wait=True)
        
drawWithPoppy(-0.20, -0.17)